In [7]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2


In [8]:
import pandas
import cv2
import numpy
import tensorflow

In [10]:
data = pandas.read_csv("inventory annotations.csv")

data

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height
0,gripper,99,77,5,27,298.png,200,150
1,gripper,99,78,5,26,300.png,200,150
2,gripper,99,78,5,26,302.png,200,150
3,gripper,1421,1093,96,406,304.png,2880,2160
4,gripper,1425,1078,84,425,306.png,2880,2160
...,...,...,...,...,...,...,...,...
135,gripper,1425,1078,84,421,568.png,2880,2160
136,gripper,1429,1078,77,421,570.png,2880,2160
137,gripper,1429,1074,80,425,572.png,2880,2160
138,gripper,1425,1082,84,421,574.png,2880,2160


In [11]:
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam


In [14]:
class Tracker():
    
    H = 512
    W = 512
    
    def __init__(self):
        pass
    
    @staticmethod
    def build_model(input_shape):
        input = layers.Input(input_shape)

        pre_trained = MobileNetV2(
            include_top = False,
            weights = "imagenet",
            input_tensor = input,
            alpha = 1.0
        )
        pre_trained.trainable = False
        x = pre_trained.output
        x = layers.Conv2D(256, kernel_size=1, padding="same")(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4, activation="sigmoid")(x)

        model = Model(input,x)
        return model

    
    def load_dataset(split = 0.2):
        data = pandas.read_csv('/Users/akshitshishodia/tracker/inventory annotations.csv')
        path  = '/Users/akshitshishodia/tracker/DATASET_/DATASET_INV/'
        image = list()
        bounding_box = list()

        for idx,rows in data.iterrows():
            name = path + str(rows['image_name'])
            bbox = [rows['bbox_x'],rows['bbox_y'],rows['bbox_width'],rows['bbox_height']]
            image.append(name)
            bounding_box.append(bbox)
        
        sample_size = int(len(image)*0.2)
        
        train_x, valid_x = train_test_split(image,test_size=sample_size,random_state=42)
        train_y, valid_y = train_test_split(bounding_box,test_size=sample_size,random_state=42)

        train_x, test_x = train_test_split(train_x,test_size=sample_size,random_state=42)
        train_y, test_y = train_test_split(train_y, test_size=sample_size, random_state=42)

        return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

    @classmethod
    def read_image(cls,path,bbox):
        path = path.decode()
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        height,width,_ = img.shape
        img = cv2.resize(img,(cls.W,cls.H))
        img = (img-127.5)/127.5
        img = img.astype(numpy.float32)

        x,y,w,h = bbox
        x_c = x+w
        y_c = y+h

        norm_x = float(x/width)
        norm_y = float(y/height)
        norm_xc = float(x_c/width)
        norm_yc = float(y_c/height)
        
        normalised = numpy.array([norm_x,norm_y,norm_xc,norm_yc],dtype = numpy.float32)

        return img, normalised
    
    @classmethod 
    def parse(cls,x,y):
        x,y = tensorflow.numpy_function(cls.read_image,[x,y],[tensorflow.float32,tensorflow.float32])
        x.set_shape([cls.H,cls.W,3])
        y.set_shape([4])
        return x,y
    
    @classmethod 
    def tensor_dataset(cls,image,b_box,batch=8):
        ds = tensorflow.data.Dataset.from_tensor_slices((image,b_box))
        ds = ds.map(cls.parse).batch(batch).prefetch(10)
        return ds

    def run(self):
        numpy.random.seed(42)
        tensorflow.random.set_seed(42)

        batch_size = 16
        lr = 1e-4
        epochs = 500

        (train_x,train_y),(valid_x,valid_y),(test_x,test_y) = self.load_dataset()
        training_dataset = self.tensor_dataset(train_x,train_y,batch = batch_size)

        validation_dataset = self.tensor_dataset(valid_x,valid_y,batch = batch_size)

        model = self.build_model((self.H,self.W,3))

        model.compile(
            loss = 'binary_crossentropy',
            optimizer = Adam(lr)
        )

        model_path = os.path.join('files','model_3.h5')
        csv_path = os.path.join('files','log_3.csv')

        callbacks = [
            ModelCheckpoint(model_path,verbose =1,save_best = True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
            CSVLogger(csv_path, append=True),
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
   
        ]

        history = model.fit(
            training_dataset,
            epochs = epochs,
            validation_data = validation_dataset,
            callbacks = callbacks
        )
        return history





In [15]:
tracker_obj = Tracker()
history = tracker_obj.run()


Epoch 1/500


[ WARN:2@146.155] global loadsave.cpp:244 findDecoder imread_('/Users/akshitshishodia/tracker/DATASET_/DATASET_INV538.png'): can't open/read file: check file path/integrity
2023-07-19 13:08:30.523179: W tensorflow/core/framework/op_kernel.cc:1816] UNKNOWN: AttributeError: 'NoneType' object has no attribute 'shape'
Traceback (most recent call last):

  File "/Users/akshitshishodia/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Users/akshitshishodia/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/zy/_wdf8zn51qq_j543bvqj82x40000gn/T/ipykernel_93796/658823430.py", line 58, in read_image
    height,width,_ = img.shape

AttributeError: 'NoneType' object has no attribute 'shape'


[ WARN:3@146.156] global loadsave.cpp:244 findDecoder imread_('/Users/akshitshishodia/tracker/DATASET_/DATAS

UnknownError: Graph execution error:

AttributeError: 'NoneType' object has no attribute 'shape'
Traceback (most recent call last):

  File "/Users/akshitshishodia/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    ret = func(*args)

  File "/Users/akshitshishodia/miniconda3/envs/tf/lib/python3.9/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/var/folders/zy/_wdf8zn51qq_j543bvqj82x40000gn/T/ipykernel_93796/658823430.py", line 58, in read_image
    height,width,_ = img.shape

AttributeError: 'NoneType' object has no attribute 'shape'


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_15761]

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)

history_frame.loc[:, ['loss', 'val_loss']].plot()

